# Index Tracking with Gurobi

This Python notebook is part of the webinar [Proven Techniques for Solving Financial Problems with Gurobi](https://www.gurobi.com/events/proven-techniques-for-solving-financial-problems-with-gurobi/).

The sequence of python code will:
1. Import stock data from yahoo finance
2. Clean up the data and change format
3. Perform an index tracking experiment

## Importing Data from YFinance

- Adjusted Stock price data for SP100 constitutents 
- Data from 2010 to 2022

In [1]:
from utils.data_import import get_sp100

# Options
FIRST_DATE  = "2020-01-01"
LAST_DATE   = "2025-01-01"
N_PROCESSES = 10
MKT_INDEX   = "^SP100"

df_prices = get_sp100(FIRST_DATE,LAST_DATE)

df_prices.head()

Fetching SP100 components
	-> got 101 tickers



c:\Users\joaov\Documents\bootcamp-ds-4ed\utils\data_import.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=FIRST_DATE, end=LAST_DATE)["Close"]
[*********************100%***********************]  102 of 102 completed

1 Failed download:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


Ticker,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WFC,WMT,XOM,^SP100
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,72.538498,69.823471,78.208038,192.514999,334.429993,44.517937,49.099998,201.031097,195.522354,94.900497,...,267.026428,160.492844,93.547050,46.150040,183.549072,43.349483,46.294529,36.508114,54.131073,1458.130005
2020-01-03,71.833282,69.160698,77.254601,192.194382,331.809998,44.173912,48.599998,199.666336,195.616455,93.748497,...,264.324188,159.356949,93.490997,45.612148,182.089310,42.887955,46.010307,36.185810,53.695885,1446.479980
2020-01-06,72.405678,69.706505,77.659370,190.939316,333.709991,44.208309,48.389999,201.198547,195.565109,95.143997,...,266.159149,157.992126,93.074486,44.988487,181.695541,42.795647,45.734688,36.112141,54.108177,1452.810059
2020-01-07,72.065140,69.308846,77.227623,186.816956,333.390015,43.958881,48.250000,199.306290,191.397934,95.343002,...,264.552399,156.794617,92.914261,44.559731,181.215347,42.319901,45.355724,35.777569,53.665337,1447.359985
2020-01-08,73.224411,69.800056,77.542427,187.183426,337.869995,44.474937,47.830002,199.457016,193.057983,94.598503,...,270.130249,158.511673,93.442924,44.466190,184.317383,42.398006,45.493523,35.654785,52.856045,1455.489990


## Cleaning and Splitting the Data

In [2]:
from utils.data_clean import clean_data

THRESH_VALID_DATA = 0.95 # defines where to cut stocks with missing data
PERC_SIZE_TRAIN = 0.75   # defines the size of train dataset (in %)

df_ret, df_train, df_test  = clean_data(
    df_prices, 
    thresh_valid_data = THRESH_VALID_DATA,
    size_train = PERC_SIZE_TRAIN
)

In [3]:
import gurobipy as gp
import pandas as pd
import numpy as np
from random import sample, seed

seed(20220209) # reproducibility

mkt_index = "^SP100"
n_assets = 20

# data from main notebook
r_it = df_train

# Gets the market index data
r_mkt = r_it[mkt_index]

# Removes market index form r_it, the "training" data, used it in the otimization problem
r_it = r_it.drop(mkt_index, axis = 1)

tickers = list(r_it.columns)

# Escolhe n ativos aleatórios para testar o modelo
sampled_tickers = sample(tickers, n_assets)
r_it = r_it[sampled_tickers]

print(r_it.head())

Ticker           RTX       ABT       AMD        MS        MA      ISRG  \
Date                                                                     
2020-01-03  0.001372 -0.012191 -0.010183 -0.016142 -0.009756 -0.006563   
2020-01-06  0.002152  0.005239 -0.004321 -0.003516  0.002663  0.004079   
2020-01-07 -0.003188 -0.005560 -0.002893 -0.001960 -0.003386 -0.021754   
2020-01-08  0.001501  0.004076 -0.008705  0.012765  0.017645 -0.001750   
2020-01-09  0.001499  0.002668  0.023834  0.009502  0.013110  0.006893   

Ticker          SBUX       HON       LLY      MDLZ       PFE       SPG  \
Date                                                                     
2020-01-03 -0.005820 -0.010675 -0.003328 -0.001657 -0.005365  0.001792   
2020-01-06 -0.007880 -0.007547  0.003719 -0.001659 -0.001285  0.005229   
2020-01-07 -0.003063  0.000563  0.001890 -0.004248 -0.003343 -0.011361   
2020-01-08  0.011609  0.000844  0.009056  0.002411  0.008000  0.006577   
2020-01-09  0.018564  0.007369  0.016

## Constrained Index Tracking

$
\begin{array}{llll}
  & \min              & \frac{1}{T} \; \sum_{t = 1}^{T} \left(\sum_{i = 1}^{I} \; w_{i} \: \times \: r_{i,t} - R_{t}\right)^2 \\
  & \text{subject to} &   \sum_{i = 1}^{I} w_{i}  = 1  \\
  &                   &   \sum_{i = 1}^{I} z_{i} \leq K \\
  &                   & w_i \geq 0 \\
  &                   & z_i \in {0, 1}
\end{array}
$

  

$
\begin{array}{lllll}
& where: \\
& \\
& w_i  &: \text{Weight of asset i in index} \\
& z_i &: \text{Binary variable (0, 1) that decides wheter asset i is in portfolio} \\
& R_{t} &: \text{Returns of tracked index (e.g. SP500) at time t} \\
& r_{i,j} &: \text{Return of asset i at time t}
\end{array}
$

In [4]:
# Create an empty model
m = gp.Model('gurobi_index_tracking')

# PARAMETERS 
max_assets = 10
# w_i: the i_th stock gets a weight w_i
w = pd.Series(m.addVars(sampled_tickers, 
                         lb = 0,
                         ub = 1,
                         vtype = gp.GRB.CONTINUOUS), 
               index=sampled_tickers)
# [NEW] z_i: the i_th stock gets a binary z_i
z = pd.Series(m.addVars(sampled_tickers,
                        vtype = gp.GRB.BINARY),
                index=sampled_tickers)

# CONSTRAINTS
# sum(w_i) = 1: portfolio budget constrain (long only)
m.addConstr(w.sum() == 1, 'port_budget')
# [NEW]  w_i <= z_i: restrictions of values of w_i so take it chose particular tickers
for i_ticker in sampled_tickers:
    m.addConstr(w[i_ticker] <= z[i_ticker], 
                f'dummy_restriction_{i_ticker}')
# [NEW] sum(z_i) <= max_assets: number of assets constraint
m.addConstr(z.sum() <= max_assets, 'max_assets_restriction')

m.update()

# eps_t = R_{i,t}*w - R_{M,t}
my_error = r_it.dot(w) - r_mkt

# set objective function, minimize the sum of squared tracking errors between portfolio and market returns
m.setObjective(
    gp.quicksum(my_error.pow(2)), 
    gp.GRB.MINIMIZE)     

# Optimize model
m.setParam('OutputFlag', 0)
m.setParam('TimeLimit', 60*5) # in secs
#m.setParam('MIPGap', 0.05) # in secs
m.optimize()

params = [i.X for i in m.getVars()]

n_assets = len(sampled_tickers)
w_hat = params[0:n_assets]
z_hat = params[n_assets:]
MIPGap = m.getAttr('MIPGap')
status = m.getAttr("Status")

print(f"Solution for w:") 

for i, i_ticker in enumerate(sampled_tickers):
    print(f"{i_ticker}:\t {w_hat[i]*100:.2f}%")

# check constraints
print(f"\nchecking constraints:")
print(f"sum(w) = {np.sum(w_hat)}")
print(f"sum(z) = {np.sum(z_hat)}")
print(f"w <= z = {w_hat <= z_hat}")
print(f"MIPGap={MIPGap}")
print(f"Status={status}")

Restricted license - for non-production use only - expires 2026-11-23
Solution for w:
RTX:	 0.00%
ABT:	 7.72%
AMD:	 5.26%
MS:	 0.00%
MA:	 9.25%
ISRG:	 0.00%
SBUX:	 0.00%
HON:	 0.00%
LLY:	 6.72%
MDLZ:	 15.52%
PFE:	 0.00%
SPG:	 0.00%
UBER:	 0.00%
GE:	 0.00%
MO:	 7.85%
AVGO:	 10.76%
BLK:	 6.86%
INTC:	 0.00%
GOOGL:	 21.54%
BAC:	 8.52%

checking constraints:
sum(w) = 1.0000000000112848
sum(z) = 10.0
w <= z = True
MIPGap=7.292567094346063e-05
Status=2


In [5]:
# check out of sample plot
import matplotlib.pyplot as plt

df_test = df_test

print(df_test.columns)
print(sampled_tickers)
df_test_mkt = df_test[mkt_index]

r_hat = df_test[sampled_tickers].dot(w_hat)

cumret_r = np.cumprod(1+ r_hat)
cumret_mkt = np.cumprod(1+ df_test_mkt)

fig, ax = plt.subplots()
ax.plot(cumret_mkt.index,
        cumret_mkt, 
       label = mkt_index)

ax.plot(cumret_r.index,
        cumret_r,
       label = f"ETF ({n_assets} assets)")

ax.legend()
ax.set_title(f'ETF and {mkt_index}')
ax.set_xlabel('')
ax.set_ylabel('Cumulative Returns')

plt.xticks(rotation = 90)

plt.show()

: 